In [109]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import pickle
import pickle5
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score   
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import folium
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, cross_val_predict
import statsmodels.api as sm

## block warning
import warnings
warnings.filterwarnings('ignore')

In [110]:
with open("../data/ut_poiV6.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,Station_Name,weekday_b,weekday_b_name,hour_b,...,near_inactivity_6H,near_charge_events_6H,service,entertainment,food,childcare,medical,education,parking,waste-management
0,928.5,2018-01-31 19:51:00,2018-02-01 11:19:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,2,Wednesday,19,...,0.0,0,0.057343,0.109796,0.073649,3.517121,0.466518,0.631771,0.021832,1.145959
1,363.5,2018-02-01 14:03:00,2018-02-01 20:06:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,3,Thursday,14,...,0.0,0,0.057343,0.109796,0.073649,3.517121,0.466518,0.631771,0.021832,1.145959
2,6828.5,2018-02-01 21:15:00,2018-02-06 15:03:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,3,Thursday,21,...,0.0,0,0.057343,0.109796,0.073649,3.517121,0.466518,0.631771,0.021832,1.145959
3,5871.5,2018-02-06 15:27:00,2018-02-10 17:18:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,1,Tuesday,15,...,0.0,0,0.057343,0.109796,0.073649,3.517121,0.466518,0.631771,0.021832,1.145959
4,1454.5,2018-02-10 18:26:00,2018-02-11 18:40:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,5,Saturday,18,...,0.0,0,0.057343,0.109796,0.073649,3.517121,0.466518,0.631771,0.021832,1.145959


In [111]:
df.shape

(12711, 71)

### Remove events

In [112]:
names = df['Station_Name'].unique()

In [113]:
for name in names:
    temp = df[df['Station_Name'] == name]
    rows = temp[temp['lag3'].isna()]
    if rows.life_time.sum() < 3*60:
        print("The station has the lagged")
        print(name)

# remove rows
df = df.dropna()

The station has the lagged
BOULDER / REC CENTER
The station has the lagged
BOULDER / ATRIUM ST1
The station has the lagged
BOULDER / ALPINE ST1
The station has the lagged
COMM VITALITY / 1400 WALNUT1
The station has the lagged
BOULDER / FACILITIES ST1
The station has the lagged
COMM VITALITY / 1500PEARL
The station has the lagged
BOULDER / JUNCTION ST1
The station has the lagged
COMM VITALITY / BOULDER JCTN
The station has the lagged
COMM VITALITY / 1100WALNUT1
The station has the lagged
BOULDER / BOULDER PARK
The station has the lagged
COMM VITALITY / 2200 BROADWAY1
The station has the lagged
BOULDER / EAST REC
The station has the lagged
BOULDERJUNCTION / JUNCTION ST1


In [114]:
df.shape

(12699, 71)

In [115]:
## Limit to only top 10 stations
temp = df.groupby(['Station_Name']).count().latitude # group the data for each station
names10 = temp.sort_values(ascending=False)[0:10]
names10 = names10.index.values

df = df[df['Station_Name'].isin(names10)]
df = df.reset_index(drop=True)

In [116]:
df.shape

(10762, 71)

## Remove outliers
Use boxplot outlier definition

In [117]:
# remove outliers
drop_index = [] # list of indexes to keep
names = df['Station_Name'].unique()

for name in names:
    temp = df[df['Station_Name'] == name]
    # Get interquantile ranges
    Q1, Q3 = temp.life_time.quantile([0.25, 0.75])
    IQR = Q3-Q1
    minimum = Q1 - 1.5*IQR
    maximum = Q3 + 1.5*IQR
    # Define observations which should be removed
    temp2 = df[(df['life_time'] < minimum) | (df['life_time'] > maximum) &
                  (df['Station_Name'] == name)]
    print("{n}: {s} ({p} %)".format(n=name, 
                                    s=temp2.shape[0], 
                                    p=round((temp2.shape[0]/temp.shape[0])*100,2)))
    # Add the indexes which should be dropped
    drop_index.extend(list(temp2.index))
    
print("\nThe total amount of lost events: {n} ({p} %)".format(n=len(drop_index), 
                                                              p=round(len(drop_index)/df.shape[0]*100,2)))

df = df.drop(drop_index)
df = df.reset_index(drop=True)

COMM VITALITY / 1104 SPRUCE1: 57 (4.4 %)
COMM VITALITY / 1000WALNUT: 48 (3.32 %)
BOULDER / REC CENTER: 40 (3.51 %)
BOULDER / BASELINE ST1: 42 (3.53 %)
BOULDER / ATRIUM ST1: 65 (6.58 %)
COMM VITALITY / 1400 WALNUT1: 38 (7.29 %)
COMM VITALITY / 1500PEARL: 41 (4.5 %)
COMM VITALITY / BOULDER JCTN: 40 (5.28 %)
COMM VITALITY / 1100WALNUT1: 39 (3.51 %)
BOULDER / N BOULDER REC 1: 29 (2.07 %)

The total amount of lost events: 439 (4.08 %)


In [118]:
df.shape

(10323, 71)

# Modeling

In [119]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [120]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    #print('MAE (Mean Absolute Error):', MAE)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    #print('MSE (Mean Squared Error):', MSE)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #print('RMSE (Root Mean Squared Error):', RMSE)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

### Prepare data

Add aggregated features for modeling purposes. 

In [121]:
def hour_label(X):
    if (X >= 7) & (X <= 10):
        return 'Morning'
    elif (X >= 11) & (X <= 14):
        return 'Midday'
    elif (X >= 15) & (X <= 18):
        return 'Afternoon'
    elif (X >= 19) or (X < 1):
        return 'Evening'
    elif (X >= 1) & (X <= 6):
        return 'Night'

In [122]:
## Add time of day and day of week
df['tod'] = df['hour_b'].apply(hour_label)

In [123]:
df = df[df['tod'] != 'Night']

Limit to 4 stations

In [124]:
station_subset = ['BOULDER / N BOULDER REC 1', 'COMM VITALITY / 1000WALNUT', 
        'COMM VITALITY / 1104 SPRUCE1', 'BOULDER / BASELINE ST1']
df_stat = df[df['Station_Name'].isin(station_subset)]

Add dummies

In [125]:
# Categorical features
columns_categorical = ['weekday_b_name','tod','hour_b','Station_Name']
df_stat = pd.get_dummies(df_stat, columns=columns_categorical, drop_first=True)

To make coding easier the names of the models are changed:
- M1: Baseline
- M5: Full

In [187]:
## M1
lt = ['life_time']
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
             'tod_Evening','tod_Midday', 'tod_Morning',
             'Station_Name_BOULDER / N BOULDER REC 1',
             'Station_Name_COMM VITALITY / 1000WALNUT',
             'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod


## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
             'tod_Evening','tod_Midday', 'tod_Morning',
             'Station_Name_BOULDER / N BOULDER REC 1',
             'Station_Name_COMM VITALITY / 1000WALNUT',
             'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + agg. tod + lag + activ.
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H']#,'service', 'entertainment', 'food','medical']

Test train split

In [188]:
split = 0.8

## Quantile regression - Linear Regression

#### Get/split data

In [189]:
df_stat = df_stat.sort_values(by=['datetime_start'])

df1 = df_stat[lt+features1]
df_train1, df_test1 = np.split(df1, [int(split*len(df1))])

df5 = df_stat[lt+features5_cat+features5_con]
df_train5, df_test5 = np.split(df5, [int(split*len(df5))])
scaler = StandardScaler()
df_train5[features5_con] = scaler.fit_transform(df_train5[features5_con])
df_test5[features5_con] = scaler.transform(df_test5[features5_con])

#### Baseline

In [190]:
quantile_labels = ['5th','25th','50th','75th','95th']
qr_M1 = pd.DataFrame(columns=quantile_labels+['true'])
qr_M1['true'] = df_test1['life_time']

In [191]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
for i in range(len(quantiles)):
    reg = sm.QuantReg(df_train1['life_time'],df_train1[features1]).fit(q=quantiles[i])
    pred = reg.predict(df_test1[features1])
    qr_M1[quantile_labels[i]] = pred

#### Full

In [192]:
quantile_labels = ['5th','25th','50th','75th','95th']
qr_M5 = pd.DataFrame(columns=quantile_labels+['true'])
qr_M5['true'] = df_test5['life_time']

In [193]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
for i in range(len(quantiles)):
    reg = sm.QuantReg(df_train5[lt],df_train5[features5_cat+features5_con]).fit(q=quantiles[i])
    pred = reg.predict(df_test5[features5_cat+features5_con])
    qr_M5[quantile_labels[i]] = pred

In [194]:
qr_M5

,5th,25th,50th,75th,95th,true
4701,9.268582,82.344892,206.956588,536.152606,1525.844540,139.5
2383,8.467028,76.659390,184.216183,453.425568,1515.206833,1162.5
4702,10.508735,86.789209,208.807318,524.650403,1515.875844,1356.5
9985,7.037257,30.203100,97.668971,234.650316,1268.177755,76.5
984,7.126313,59.706734,164.462950,703.669272,1376.403304,149.5
...,...,...,...,...,...,...
10321,12.210338,57.224683,138.910172,400.165266,1436.224381,66.5
1236,16.895307,114.587903,251.029090,700.689448,1773.022015,99.5
1237,11.192926,73.515270,168.060709,690.343651,1402.268158,1411.5
10322,6.363855,20.562947,72.038124,425.927768,1101.425834,1033.5


### Evaluate predictions

In [152]:
y_test1 = df_test1['life_time']

In [153]:
def ICP(q1, q2, true):
    if (true >= q1) and (true <= q2):
        return 1
    else:
        return 0

In [154]:
#### 5%-95%
## M1
qr_M1['MIL5-95'] = qr_M1['95th']-qr_M1['5th']
qr_M1 = qr_M1.reset_index()
# Get ICP
temp = []
for i in range(len(y_test1)):
    row = qr_M1.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M1['ICP5-95'] = temp

## M5
qr_M5['MIL5-95'] = qr_M5['95th']-qr_M5['5th']
qr_M5 = qr_M5.reset_index()
# Get ICP
temp = []
for i in range(len(y_test1)):
    row = qr_M5.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M5['ICP5-95'] = temp


#### 25%-75%
## M1
qr_M1['MIL25-75'] = qr_M1['75th']-qr_M1['25th']
# Get ICP
temp = []
for i in range(len(y_test1)):
    row = qr_M1.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M1['ICP25-75'] = temp

## M5
qr_M5['MIL25-75'] = qr_M5['75th']-qr_M5['25th']
# Get ICP
temp = []
for i in range(len(y_test1)):
    row = qr_M5.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M5['ICP25-75'] = temp


In [155]:
## merge the TOD to the dataframe
qr_M1 = qr_M1.set_index('index')
qr_M1 = qr_M1.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M5 = qr_M5.set_index('index')
qr_M5 = qr_M5.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)

In [157]:
print("Baseline model")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M1[qr_M1['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Baseline model
Morning:
 MIL(5-95): 1338.0 ICP(5-95): 0.902   MIL(25-75): 370.0 ICP(25-75): 0.477
Midday:
 MIL(5-95): 1419.0 ICP(5-95): 0.872   MIL(25-75): 431.0 ICP(25-75): 0.426
Afternoon:
 MIL(5-95): 1078.0 ICP(5-95): 0.8   MIL(25-75): 428.0 ICP(25-75): 0.386
Evening:
 MIL(5-95): 1903.0 ICP(5-95): 0.902   MIL(25-75): 408.0 ICP(25-75): 0.627


In [158]:
print("Full model")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M5[qr_M5['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Full model
Morning:
 MIL(5-95): 1218.0 ICP(5-95): 0.917   MIL(25-75): 409.0 ICP(25-75): 0.485
Midday:
 MIL(5-95): 1401.0 ICP(5-95): 0.87   MIL(25-75): 399.0 ICP(25-75): 0.455
Afternoon:
 MIL(5-95): 1388.0 ICP(5-95): 0.834   MIL(25-75): 703.0 ICP(25-75): 0.41
Evening:
 MIL(5-95): 1605.0 ICP(5-95): 0.882   MIL(25-75): 349.0 ICP(25-75): 0.565


In [159]:
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M1['MIL5-95']),2), 
                                            round(np.sum(qr_M1['ICP5-95'])/len(qr_M1),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M5['MIL5-95']),2), 
                                            round(np.sum(qr_M5['ICP5-95'])/len(qr_M5),3)))

print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M1['MIL25-75']),2), 
                                            round(np.sum(qr_M1['ICP25-75'])/len(qr_M1),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M5['MIL25-75']),2), 
                                            round(np.sum(qr_M5['ICP25-75'])/len(qr_M5),3)))


Base (5%-95%): MIL: 1432.62 ICP: 0.863
Full (5%-95%): MIL: 1424.51 ICP: 0.869
Base (25%-75%): MIL: 416.75 ICP: 0.472
Full (25%-75%): MIL: 474.02 ICP: 0.474


Get regression metrics for the 50th percentile

In [160]:
results = {}

In [161]:
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1['true'],qr_M1['50th'])
results['M1'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1['true'],qr_M1['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5['true'],qr_M5['50th'])
results['M5'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5['true'],qr_M5['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [162]:
results

{'M1': {'RMSE_test': 493.7849988438025,
  'MAE_test': 300.1897648873799,
  'rsq_test': -0.06798732172293254,
  'MAPE_test': 1116.366377440992,
  'NRMSE_test': 1.0694011604591802},
 'M5': {'RMSE_test': 481.6415381856231,
  'MAE_test': 297.19256142219837,
  'rsq_test': -0.016104050546759563,
  'MAPE_test': 1127.5307608131125,
  'NRMSE_test': 1.0431017974767998}}